In [ ]:
%matplotlib notebook
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

from itertools import chain, combinations
import matplotlib.pyplot as plt
import numpy as np
import functools

import sys
sys.path.append('../')
from dyrect import Poset, Complex, draw_complex

In [ ]:
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(1, len(s)+1))

In [ ]:
### Create a poset representing (n-1)-simplex
n = 6
sims = list(powerset(range(n)))

nsims = len(sims)
bmat = np.zeros((nsims, nsims))

### hash maps:
## simplex index to simplex (i.e., collection of vertices)
idx2sim = dict()
## simplex its index
sim2idx = dict()

for i, s in enumerate(sims):
    sim2idx[s] = i
    idx2sim[i] = s
    
    if len(s) == 1:
        continue
    for face in combinations(s, len(s)-1):
#         print(s, face)
        bmat[i, sim2idx[face]] = 1
poset = Poset.from_dag(bmat)

In [ ]:
### Coordinates for the cycle we will patch later, there should be at least n points in the list
verts = np.array([[0.0, 0.0], [1.0, 0.0], [1.0, 1.0], [0.8, 0.2], [0.5, 0.2], [0.45, 0.31]])
# verts = np.array([[0.0, 0.0, 0.0], [1.0, 0.0, 0.0], [1.0, 1.0, .2], [0.8, 0.2, 0.8], [0.5, 0.2, 0.6], [0.2, 0.75, 0.4], [0.25, 0.4, 0.2]])
# verts = np.array([[0.0, 0.0], [1.0, 0.0], [0.0, 0.5], [0.25, 0.3], [0.45, 0.2], [0.2, 0.75]])

### hash maps:
## simplex index to its diameter
idx2diam = dict()
## simplex index to the sorted list of diameters of the simplex edges
idx2edges = dict()
for i in range(nsims):
    if len(idx2sim[i]) < 2:
        idx2diam[i] = 0
        idx2edges[i] = [0]
    elif len(idx2sim[i]) == 2:
        edge = idx2sim[i]
        idx2diam[i] = np.linalg.norm(verts[edge[0]]-verts[edge[1]])
        idx2edges[i] = [idx2diam[i]]
    else:
        mouth = poset.mouth([i])
        idx2diam[i] = max([idx2diam[s] for s in mouth])
        simplex = idx2sim[i]
        idx2edges[i] = sorted([idx2diam[sim2idx[s]] for s in combinations(simplex, 2)], reverse=True)


In [ ]:
def diamcheck(x, y):
    """ given indices of two simplices x and y, check which:
        1) has higher dimension
        2) has longer edges
    """
    dx = idx2edges[x]
    dy = idx2edges[y]
    if len(dx) > len(dy):
        return 1
    elif len(dx) < len(dy):
        return -1
    else:
        for i in range(len(dx)):
            if dx[i] > dy[i]:
                return 1
            elif dx[i] < dy[i]:
                return -1
    return 0

In [ ]:
### Reduction algorithm

### sequence of edges, i.e. initial cycle to be patched
cedges = [(i-1, i) for i in range(1, n)] + [(0, n-1)]

### above cycle, but as a subcomplex, i.e. set of indices of edges plus vertices
cycle = set(range(n)).union(set([sim2idx[i] for i in cedges]))

### filling of the circle to be constructed
filling = set(range(nsims)).difference(cycle)
print(cycle)

while True:
    old_filling = filling

    ### sorting of simplices for reduction with respect to:
    ## #1 method diamcheck
    sim_queue = sorted(list(filling), key=functools.cmp_to_key(diamcheck), reverse=True)
    ## #2 diameter of a simplex
#     sim_queue = sorted(list(filling), key=lambda x: idx2diam[x], reverse=True)
    ## #3 without sorting
#     sim_queue = reversed(list(filling))
#     print(sim_queue)
    for i in sim_queue:
        up = (poset.above(i)).intersection(filling)
        if len(up) == 2:
            filling = filling.difference(up)
#             print(i, up, [idx2sim[x] for x in up], filling)
            break
    if old_filling == filling:
        print(True)
        break
#     break

    

In [ ]:
space_dimension = 2

patch = [idx2sim[i] for i in filling.union(cycle)]
cpatch = [idx2sim[i] for i in cycle]
pcomp = {0:[], 1:[], 2:[]}
cycomp = {0:[], 1:[], 2:[]}
for p in patch:
    pcomp[len(p)-1].append(p)
for p in cpatch:
    cycomp[len(p)-1].append(p)
    
print(pcomp)
# verts = np.array([[0.0, 0.0], [1.0, 0.0], [1.0, 1.0], [0.8, 0.5], [0.6, 0.2], [0.7, 0.75]])
comp = Complex.construct(pcomp, verts, max_dim=2)
comp2 = Complex.construct(cycomp, verts, max_dim=2)
# print(comp.coordinates)
draw_complex(comp, dim=space_dimension)
draw_complex(comp2, dim=space_dimension)

## Patching 2

In [ ]:
### this is unfinished

cverts = set(range(0, n))
cedges = [(i-1, i) for i in range(1, n)] + [(0, n-1)]
# print([sim2idx[i] for i in cedges])
cycle = set(range(n)).union(set([sim2idx[i] for i in cedges]))
filling = list(cycle)

print(cedges, filling)
ncverts = cverts
nedges = cedges

min_edge = np.infty
new_edge = -1
for e in set(combinations(ncverts, 2)).difference(set(nedges)):
#     print(e, idx2diam[sim2idx[e]])
    if idx2diam[sim2idx[e]] < min_edge:
        neighs0 = [set(ce) for ce in nedges if e[0] in ce]
        neighs1 = [set(ce) for ce in nedges if e[1] in ce]
        nn = (set.union(*neighs0)).intersection(set.union(*neighs1))
        if len(nn) > 0:
            min_edge = idx2diam[sim2idx[e]]
            new_edge = e
        print(e, neighs0, neighs1, nn)
print(new_edge)
ncverts = ncverts.difference(nn)
nedges = nedges + [new_edge]
print(ncverts, nedges)
[sim2idx[tuple(nn)], sim2idx[new_edge], sim2idx[tuple(sorted(list(nn) + list(new_edge)))]]
filling = filling +[sim2idx[new_edge], sim2idx[tuple(sorted(list(nn) + list(new_edge)))]]
filling